<a href="https://colab.research.google.com/github/fgkindel/fgkindel/blob/main/Copy_of_Archive_of_the_Nots_Lore_Garden_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# The Archive of the Nots: Lore Garden System
# Copy this to a new Google Colab notebook

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import random
import datetime
import time
import json
import requests
from google.colab import auth
import gspread
from oauth2client.client import GoogleCredentials
from google.colab import output
import seaborn as sns
from IPython.display import HTML, display, clear_output

# Set up visualization styles
plt.style.use('dark_background')
sns.set_style("darkgrid")

# Configure color scheme
COLORS = {
    'seed': '#00C853',      # Green for seeds
    'sprout': '#FFEB3B',    # Yellow for sprouts
    'growing': '#5D5CDE',   # Purple for growing
    'flowering': '#FF4081', # Pink for flowering
    'connected': '#90CAF9', # Light blue for connections
    'background': '#0A0A12' # Dark background
}

# ================ SETUP ================

def setup_sheets_connection():
    """Authenticate and connect to Google Sheets."""
    auth.authenticate_user()
    gc = gspread.authorize(GoogleCredentials.get_application_default())
    return gc

def create_or_get_sheets():
    """Create or get the necessary sheets for the lore garden."""
    gc = setup_sheets_connection()

    # Get or create the spreadsheet
    try:
        # Try to open existing spreadsheet
        spreadsheet = gc.open("The Archive of the Nots - Lore Garden")
        print("Connected to existing Lore Garden spreadsheet.")
    except:
        # Create new spreadsheet if it doesn't exist
        spreadsheet = gc.create("The Archive of the Nots - Lore Garden")
        print("Created new Lore Garden spreadsheet.")

    # Make sure all required worksheets exist
    sheet_names = [sheet.title for sheet in spreadsheet.worksheets()]

    # 1. Fragments Sheet
    if 'Fragments' not in sheet_names:
        fragments_sheet = spreadsheet.add_worksheet(title='Fragments', rows=1000, cols=15)
        # Add headers
        fragments_sheet.update('A1:K1', [['fragment_id', 'content', 'growth_stage',
                                       'planted_by', 'planted_date', 'last_watered',
                                       'water_count', 'connection_count', 'view_count',
                                       'theme_tags', 'enriched_content']])
        print("Created Fragments sheet.")
    else:
        fragments_sheet = spreadsheet.worksheet('Fragments')
        print("Connected to existing Fragments sheet.")

    # 2. Connections Sheet
    if 'Connections' not in sheet_names:
        connections_sheet = spreadsheet.add_worksheet(title='Connections', rows=1000, cols=5)
        connections_sheet.update('A1:D1', [['connection_id', 'source_id', 'target_id', 'created_date']])
        print("Created Connections sheet.")
    else:
        connections_sheet = spreadsheet.worksheet('Connections')
        print("Connected to existing Connections sheet.")

    # 3. Growth History Sheet
    if 'Growth_History' not in sheet_names:
        history_sheet = spreadsheet.add_worksheet(title='Growth_History', rows=1000, cols=6)
        history_sheet.update('A1:E1', [['history_id', 'fragment_id', 'from_stage', 'to_stage', 'date']])
        print("Created Growth_History sheet.")
    else:
        history_sheet = spreadsheet.worksheet('Growth_History')
        print("Connected to existing Growth_History sheet.")

    # 4. Meta Sheet (for overall mythology)
    if 'Meta' not in sheet_names:
        meta_sheet = spreadsheet.add_worksheet(title='Meta', rows=100, cols=5)
        meta_sheet.update('A1:D1', [['key', 'value', 'updated_date', 'notes']])
        # Initialize with some metadata
        meta_sheet.update('A2:D5', [
            ['dominant_themes', '[]', datetime.datetime.now().isoformat(), 'Currently dominant themes in the garden'],
            ['mythology_synthesis', '', datetime.datetime.now().isoformat(), 'Current state of the mythology'],
            ['fragment_count', '0', datetime.datetime.now().isoformat(), 'Total number of fragments'],
            ['garden_stage', 'seedling', datetime.datetime.now().isoformat(), 'Overall garden development stage']
        ])
        print("Created Meta sheet.")
    else:
        meta_sheet = spreadsheet.worksheet('Meta')
        print("Connected to existing Meta sheet.")

    return {
        'spreadsheet': spreadsheet,
        'fragments': fragments_sheet,
        'connections': connections_sheet,
        'history': history_sheet,
        'meta': meta_sheet
    }

# ================ FRAGMENT FUNCTIONS ================

def plant_fragment(content, planted_by="Anonymous", theme_tags=None):
    """Plant a new fragment in the garden (create new entry)."""
    sheets = create_or_get_sheets()
    fragments_sheet = sheets['fragments']
    meta_sheet = sheets['meta']

    # Get all existing fragments to determine next ID
    all_fragments = fragments_sheet.get_all_values()
    next_id = len(all_fragments)
    if next_id == 1:  # Only header row exists
        next_id = 1
    else:
        next_id = int(all_fragments[-1][0]) + 1 if len(all_fragments) > 1 else 1

    # Format tags
    if theme_tags and isinstance(theme_tags, list):
        theme_tags = json.dumps(theme_tags)
    elif not theme_tags:
        theme_tags = json.dumps(["untagged"])

# The Archive of the Nots: Lore Garden System
# Copy this to a new Google Colab notebook

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import random
import datetime
import time
import json
import requests
from google.colab import auth
import gspread
from oauth2client.client import GoogleCredentials
from google.colab import output
import seaborn as sns
from IPython.display import HTML, display, clear_output

# Set up visualization styles
plt.style.use('dark_background')
sns.set_style("darkgrid")

# Configure color scheme
COLORS = {
    'seed': '#00C853',      # Green for seeds
    'sprout': '#FFEB3B',    # Yellow for sprouts
    'growing': '#5D5CDE',   # Purple for growing
    'flowering': '#FF4081', # Pink for flowering
    'connected': '#90CAF9', # Light blue for connections
    'background': '#0A0A12' # Dark background
}

# ================ SETUP ================

def setup_sheets_connection():
    """Authenticate and connect to Google Sheets."""
    auth.authenticate_user()
    gc = gspread.authorize(GoogleCredentials.get_application_default())
    return gc

def create_or_get_sheets():
    """Create or get the necessary sheets for the lore garden."""
    gc = setup_sheets_connection()

    # Get or create the spreadsheet
    try:
        # Try to open existing spreadsheet
        spreadsheet = gc.open("The Archive of the Nots - Lore Garden")
        print("Connected to existing Lore Garden spreadsheet.")
    except:
        # Create new spreadsheet if it doesn't exist
        spreadsheet = gc.create("The Archive of the Nots - Lore Garden")
        print("Created new Lore Garden spreadsheet.")

    # Make sure all required worksheets exist
    sheet_names = [sheet.title for sheet in spreadsheet.worksheets()]

    # 1. Fragments Sheet
    if 'Fragments' not in sheet_names:
        fragments_sheet = spreadsheet.add_worksheet(title='Fragments', rows=1000, cols=15)
        # Add headers
        fragments_sheet.update('A1:K1', [['fragment_id', 'content', 'growth_stage',
                                       'planted_by', 'planted_date', 'last_watered',
                                       'water_count', 'connection_count', 'view_count',
                                       'theme_tags', 'enriched_content']])
        print("Created Fragments sheet.")
    else:
        fragments_sheet = spreadsheet.worksheet('Fragments')
        print("Connected to existing Fragments sheet.")

    # 2. Connections Sheet
    if 'Connections' not in sheet_names:
        connections_sheet = spreadsheet.add_worksheet(title='Connections', rows=1000, cols=5)
        connections_sheet.update('A1:D1', [['connection_id', 'source_id', 'target_id', 'created_date']])
        print("Created Connections sheet.")
    else:
        connections_sheet = spreadsheet.worksheet('Connections')
        print("Connected to existing Connections sheet.")

    # 3. Growth History Sheet
    if 'Growth_History' not in sheet_names:
        history_sheet = spreadsheet.add_worksheet(title='Growth_History', rows=1000, cols=6)
        history_sheet.update('A1:E1', [['history_id', 'fragment_id', 'from_stage', 'to_stage', 'date']])
        print("Created Growth_History sheet.")
    else:
        history_sheet = spreadsheet.worksheet('Growth_History')
        print("Connected to existing Growth_History sheet.")

    # 4. Meta Sheet (for overall mythology)
    if 'Meta' not in sheet_names:
        meta_sheet = spreadsheet.add_worksheet(title='Meta', rows=100, cols=5)
        meta_sheet.update('A1:D1', [['key', 'value', 'updated_date', 'notes']])
        # Initialize with some metadata
        meta_sheet.update('A2:D5', [
            ['dominant_themes', '[]', datetime.datetime.now().isoformat(), 'Currently dominant themes in the garden'],
            ['mythology_synthesis', '', datetime.datetime.now().isoformat(), 'Current state of the mythology'],
            ['fragment_count', '0', datetime.datetime.now().isoformat(), 'Total number of fragments'],
            ['garden_stage', 'seedling', datetime.datetime.now().isoformat(), 'Overall garden development stage']
        ])
        print("Created Meta sheet.")
    else:
        meta_sheet = spreadsheet.worksheet('Meta')
        print("Connected to existing Meta sheet.")

    return {
        'spreadsheet': spreadsheet,
        'fragments': fragments_sheet,
        'connections': connections_sheet,
        'history': history_sheet,
        'meta': meta_sheet
    }

# ================ FRAGMENT FUNCTIONS ================

def plant_fragment(content, planted_by="Anonymous", theme_tags=None):
    """Plant a new fragment in the garden (create new entry)."""
    sheets = create_or_get_sheets()
    fragments_sheet = sheets['fragments']
    meta_sheet = sheets['meta']

    # Get all existing fragments to determine next ID
    all_fragments = fragments_sheet.get_all_values()
    next_id = len(all_fragments)
    if next_id == 1:  # Only header row exists
        next_id = 1
    else:
        next_id = int(all_fragments[-1][0]) + 1 if len(all_fragments) > 1 else 1

    # Format tags
    if theme_tags and isinstance(theme_tags, list):
        theme_tags = json.dumps(theme_tags)
    elif not theme_tags:
        theme_tags = json.dumps(["untagged"])

    # Prepare new fragment data
    now = datetime.datetime.now().isoformat()
    new_fragment = [
        next_id,                # fragment_id
        content,                # content
        'seed',                 # growth_stage
        planted_by,             # planted_by
        now,                    # planted_date
        now,                    # last_watered
        1,                      # water_count
        0,                      # connection_count
        0,                      # view_count
        theme_tags,             # theme_tags
        ""                      # enriched_content
    ]

    # Append the new fragment to the sheet
    fragments_sheet.append_row(new_fragment)

    # Update fragment count in Meta sheet
    current_fragment_count = int(meta_sheet.cell(4,2).value)
    meta_sheet.update_cell(4, 2, current_fragment_count + 1)

    print(f"Fragment '{content[:20]}...' planted successfully (ID: {next_id})")
        'seed',                 # growth_stage
        planted_by,             # planted_by
        now,                    # planted_date
        now,                    # last_watered
        1,                      # water_count
        0,                      # connection_count
# The Archive of the Nots: Lore Garden System
# Copy this to a new Google Colab notebook

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import random
import datetime
import time
import json
import requests
from google.colab import auth
import gspread
from oauth2client.client import GoogleCredentials
from google.colab import output
import seaborn as sns
from IPython.display import HTML, display, clear_output

# Set up visualization styles
plt.style.use('dark_background')
sns.set_style("darkgrid")

# Configure color scheme
COLORS = {
    'seed': '#00C853',      # Green for seeds
    'sprout': '#FFEB3B',    # Yellow for sprouts
    'growing': '#5D5CDE',   # Purple for growing
    'flowering': '#FF4081', # Pink for flowering
    'connected': '#90CAF9', # Light blue for connections
    'background': '#0A0A12' # Dark background
}

# ================ SETUP ================

def setup_sheets_connection():
    """Authenticate and connect to Google Sheets."""
    auth.authenticate_user()
    gc = gspread.authorize(GoogleCredentials.get_application_default())
    return gc

def create_or_get_sheets():
    """Create or get the necessary sheets for the lore garden."""
    gc = setup_sheets_connection()

    # Get or create the spreadsheet
    try:
        # Try to open existing spreadsheet
        spreadsheet = gc.open("The Archive of the Nots - Lore Garden")
        print("Connected to existing Lore Garden spreadsheet.")
    except:
        # Create new spreadsheet if it doesn't exist
        spreadsheet = gc.create("The Archive of the Nots - Lore Garden")
        print("Created new Lore Garden spreadsheet.")

    # Make sure all required worksheets exist
    sheet_names = [sheet.title for sheet in spreadsheet.worksheets()]

    # 1. Fragments Sheet
    if 'Fragments' not in sheet_names:
        fragments_sheet = spreadsheet.add_worksheet(title='Fragments', rows=1000, cols=15)
        # Add headers
        fragments_sheet.update('A1:K1', [['fragment_id', 'content', 'growth_stage',
                                       'planted_by', 'planted_date', 'last_watered',
                                       'water_count', 'connection_count', 'view_count',
                                       'theme_tags', 'enriched_content']])
        print("Created Fragments sheet.")
    else:
        fragments_sheet = spreadsheet.worksheet('Fragments')
        print("Connected to existing Fragments sheet.")

    # 2. Connections Sheet
    if 'Connections' not in sheet_names:
        connections_sheet = spreadsheet.add_worksheet(title='Connections', rows=1000, cols=5)
        connections_sheet.update('A1:D1', [['connection_id', 'source_id', 'target_id', 'created_date']])
        print("Created Connections sheet.")
    else:
        connections_sheet = spreadsheet.worksheet('Connections')
        print("Connected to existing Connections sheet.")

    # 3. Growth History Sheet
    if 'Growth_History' not in sheet_names:
        history_sheet = spreadsheet.add_worksheet(title='Growth_History', rows=1000, cols=6)
        history_sheet.update('A1:E1', [['history_id', 'fragment_id', 'from_stage', 'to_stage', 'date']])
        print("Created Growth_History sheet.")
    else:
        history_sheet = spreadsheet.worksheet('Growth_History')
        print("Connected to existing Growth_History sheet.")

    # 4. Meta Sheet (for overall mythology)
    if 'Meta' not in sheet_names:
        meta_sheet = spreadsheet.add_worksheet(title='Meta', rows=100, cols=5)
        meta_sheet.update('A1:D1', [['key', 'value', 'updated_date', 'notes']])
        # Initialize with some metadata
        meta_sheet.update('A2:D5', [
            ['dominant_themes', '[]', datetime.datetime.now().isoformat(), 'Currently dominant themes in the garden'],
            ['mythology_synthesis', '', datetime.datetime.now().isoformat(), 'Current state of the mythology'],
            ['fragment_count', '0', datetime.datetime.now().isoformat(), 'Total number of fragments'],
            ['garden_stage', 'seedling', datetime.datetime.now().isoformat(), 'Overall garden development stage']
        ])
        print("Created Meta sheet.")
    else:
        meta_sheet = spreadsheet.worksheet('Meta')
        print("Connected to existing Meta sheet.")

    return {
        'spreadsheet': spreadsheet,
        'fragments': fragments_sheet,
        'connections': connections_sheet,
        'history': history_sheet,
        'meta': meta_sheet
    }

# ================ FRAGMENT FUNCTIONS ================

def plant_fragment(content, planted_by="Anonymous", theme_tags=None):
    """Plant a new fragment in the garden (create new entry)."""
    sheets = create_or_get_sheets()
    fragments_sheet = sheets['fragments']
    meta_sheet = sheets['meta']

    # Get all existing fragments to determine next ID
    all_fragments = fragments_sheet.get_all_values()
    next_id = len(all_fragments)
    if next_id == 1:  # Only header row exists
        next_id = 1
    else:
        next_id = int(all_fragments[-1][0]) + 1 if len(all_fragments) > 1 else 1

    # Format tags
    if theme_tags and isinstance(theme_tags, list):
        theme_tags = json.dumps(theme_tags)
    elif not theme_tags:
        theme_tags = json.dumps(["untagged"])

    # Prepare new fragment data
    now = datetime.datetime.now().isoformat()
    new_fragment = [
        next_id,                # fragment_id
        content,                # content
        'seed',                 # growth_stage
        planted_by,             # planted_by
        now,                    # planted_date
        now,                    # last_watered
        1,                      # water_count
        0,                      # connection_count
        0,                      # view_count
        theme_tags,             # theme_tags
        ""                      # enriched_content
    ]

    # Append the new fragment to the sheet
    fragments_sheet.append_row(new_fragment)

    # Update fragment count in Meta sheet
    current_fragment_count = int(meta_sheet.cell(4,2).value)
    meta_sheet.update_cell(4, 2, current_fragment_count + 1)

    print(f"Fragment '{content[:20]}...' planted successfully (ID: {next_id})")

SyntaxError: '[' was never closed (<ipython-input-1-e3ec73f09119>, line 138)

In [3]:
def setup_sheets_connection():
    """Authenticate and connect to Google Sheets."""
    auth.authenticate_user()
    gc = gspread.authorize(GoogleCredentials.get_application_default())
    return gc

def create_or_get_sheets():
    """Create or get the necessary sheets for the lore garden."""
    gc = setup_sheets_connection()

    # ... (Code to create/get spreadsheet and worksheets) ...

    return {
        'spreadsheet': spreadsheet,
        'fragments': fragments_sheet,
        'connections': connections_sheet,
        'history': history_sheet,
        'meta': meta_sheet
    }

In [2]:
def plant_fragment(content, planted_by="Anonymous", theme_tags=None):
    """Plant a new fragment in the garden (create new entry)."""
    sheets = create_or_get_sheets()
    fragments_sheet = sheets['fragments']
    meta_sheet = sheets['meta']

    # Get all existing fragments to determine next ID
    all_fragments = fragments_sheet.get_all_values()
    next_id = len(all_fragments)
    if next_id == 1:  # Only header row exists
        next_id = 1
    else:
        next_id = int(all_fragments[-1][0]) + 1 if len(all_fragments) > 1 else 1

    # Format tags
    if theme_tags and isinstance(theme_tags, list):
        theme_tags = json.dumps(theme_tags)
    elif not theme_tags:
        theme_tags = json.dumps(["untagged"])

    # Prepare new fragment data
    now = datetime.datetime.now().isoformat()
    new_fragment = [
        next_id,                # fragment_id
        content,                # content
        'seed',                 # growth_stage
        planted_by,             # planted_by
        now,                    # planted_date
        now,                    # last_watered
        1,                      # water_count
        0,                      # connection_count
        0,                      # view_count
        theme_tags,             # theme_tags
        ""                      # enriched_content
    ] # Closing bracket added here

    # Append the new fragment to the sheet
    fragments_sheet.append_row(new_fragment)

    # Update fragment count in Meta sheet
    current_fragment_count = int(meta_sheet.cell(4,2).value)
    meta_sheet.update_cell(4, 2, current_fragment_count + 1)

    print(f"Fragment '{content[:20]}...' planted successfully (ID: {next_id})")